# Lab 9
<br>

# More on OOP
---

##### CS1P. Semester 2. Python 3.x
 ---

## Purpose of this lab

After doing this lab, you will:
* get more practice working with files
* get more practice using recursion and functional approaches
* improve on your problem solving and program planning skills
* have a better understanding of how to design with OOP principles
* have experience in building small OOP programs

# A. Closely connected stations

You have a set of data on train stations in the UK. The train station information is stored in two files. 

`stations.txt` has the name of each station and its three letter identification code ("station ID"):

    Partick,PTK
    Glasgow Queen Street,GLQ
    London Kings Cross,KGX
    ...
    
Each station name is separated from the station ID with a comma. Station names and station IDs never have commas in them.


`connections.txt` consists of every connection between two stations, along with its distance in miles in the format `station_a,station_b,miles`:

    PTK,GLQ,4.2
    PTK,MLN,8.1
    PTK,BRN,6.3
    ...
    GLQ,EDB,42

## The problem

Two stations are considered "closely connected" if one can be reached from the other either directly or with at most one intermediate station. For example, in the above data:

*    PTK is closely connected to GLQ as they have a direct connection
*    GLQ is closely connected to MLN as they are connected via PTK
*    KGX is **not** closely connected to MLN as *two* intermediate stations would be needed (KGX->GLQ->PTK->MLN)

You task is to write a class that will take a station ID from a user (e.g. "PTK") and then print out a report of *all* closely connected stations, using their proper names. The output should include the *total* distance along the route. For example:

    Station ID? PTK
    Partick (PTK) is closely connected to:
         Glasgow Queen Street (GLQ), distance 4.2 miles
         Bearsden (BRN), distance 6.3 miles
         Milngavie (MLN), distance 8.1 miles
         Edinburgh (EDB), distance 46.2 miles
         London Kings Cross (KGX), distance 443.4 miles

    
You should use the following information:

* You can assume that every station is connected to at least one other station. 
* Your results should be ordered by distance, shortest route first. 
* If a station is connected in many ways to another station, print out the shortest route only. You should **not** print a route from a station back to itself (e.g. PTK->BRN->PTK).
* The file *only* lists connections in one direction, but travel is possible in both directions (for example you can go PTK->GLQ or GLQ->PTK, but only PTK->GLQ is listed in this file). Your solution should show the GLQ->PTK route.
* You can add more stations and connections to the file.
* Make your program robust by detecting and handling errors. Think of what could go wrong.


<div class="alert alert-info">
You will need to carefully think of how to structure your class. What are the instance variables? What parameters do you need to pass to the constructor method? What are your instance methods?
    
</div>

In [1]:
class Station:
    def __init__(self, name, code):
        self.name = name
        self.code = code
        self.connections = {}
    
    def add_connection(self, code, distance):
        self.connections[code] = distance
    
    def get_connections(self):
        return self.connections
    
    def __str__(self):
        return f"{self.code}:{self.name}"
                
def read_stations(filename):
    stations = {}
    with open(filename) as f:
        l = f.read().splitlines()
    for elt in l:
        line = elt.split(',')
        stations[line[1]]=line[0]
    return stations
    
def add_connection(code, code_2, distance, connections, stations):
    if code not in connections.keys():
        connections[code] = Station(stations[code], code)
        connections[code].add_connection(code_2, distance)
    else:
        connections[code].add_connection(code_2, distance)
    return connections

def close_connections_algorithm(connections):
    outer_dic = {}
    for station in connections:
        inner_dic = {}
        for con in connections[station].get_connections():
            for close_con in connections[con].get_connections():
                if close_con != connections[station].code:
                    sum_distance = connections[station].get_connections()[con] + connections[con].get_connections()[close_con]
                    if close_con in connections[station].get_connections():
                        if sum_distance < connections[station].get_connections()[close_con]:
                            inner_dic[close_con] = sum_distance
                    else:
                        inner_dic[close_con] = sum_distance

        outer_dic[station] = inner_dic
        inner_dic = {}
    return outer_dic

def read_connections(filename, stations):
    connections = {}
    with open(filename) as f:
        l = f.read().splitlines()
    for elt in l:
        line = elt.split(',')
        station_1_code = line[0]
        station_2_code = line[1]
        distance = float(line[2])
        
        connections = add_connection(station_1_code, station_2_code, distance, connections, stations)
        connections = add_connection(station_2_code, station_1_code, distance, connections, stations)
        
    outer_dic = close_connections_algorithm(connections)
    
    for station in outer_dic:
        for close_station in outer_dic[station]:
            connections[station].add_connection(close_station, outer_dic[station][close_station])
            
    return connections

def print_station_connections(code, connections, stations):
        print(f"{stations[code]}({code}) is closely connected to:")
        for elt in dict(sorted(connections[code].get_connections().items(), key=lambda item: item[1])):
            print(f"\t{stations[elt]} ({elt}), distance " + "{:.1f}".format(connections[code].get_connections()[elt]) + " miles")
        


def input_station():
    station_code = input("Enter station ID: ")
    
    stations = read_stations("stations.txt")
    connections = read_connections("connections.txt", stations)
    
    print_station_connections(station_code, connections, stations)
    
#main
input_station()



Enter station ID: PTK
Partick(PTK) is closely connected to:
	Glasgow Queen Street (GLQ), distance 4.2 miles
	Bearsden (BRN), distance 6.3 miles
	Milngavie (MLN), distance 8.1 miles
	Edinburgh (EDB), distance 46.2 miles
	London Kings Cross (KGX), distance 443.4 miles


# B. Family tree

We will work with *family tree*, a data source representing the (fictional) genealogical history of a small town. We'll work with a *very* simplified form of this problem, where we only see a record of births and deaths. 

## A person

Let's model a person with a new class, `Person`. We can do noun-verb analysis: we identify **nouns** (in bold) as possible instance variables, and *verbs* (in italics) as possible instance methods.

> * A Person has a **name**. They have a **birth year** and a **death year**. They have a **mother** and a **father**. They have a **unique ID**.
> * We can *ask what someone's age was in a particular year*
> * We can *ask whether someone was alive in a particular year*

## B.1 
Create a class `Person` with appropriate instance variables and methods, given this description. Further, add a `__repr__` method that returns the person's name and birth year as a string. 

**NOTE**: `__repr__` method is similar to the `__str__` method, they are both used for string representation in Python. The difference between them, going by the official Python documentation, is that `__str__` is used to find the "informal" (readable to the user) string representation of an object whereas `__repr__` is used to find the "official" string representation of an object.

**NOTE**: To solve this problem, you will have to first design the structure of the class! 

In [10]:
## Solution
class Person:
    def __init__(self, unique_id, name, birth_year, death_year, mother_id, father_id):
        self.unique_id = unique_id
        self.name = name
        self.birth_year = birth_year
        self.death_year = death_year
        self.mother_id = mother_id
        self.father_id = father_id
        
    def age_according_to_year(self, year):
        return year - int(self.birth_year)
    
    def is_alive_in_particular_year(self, year):
        flag = False
        if self.birth_year != "-" :
            if self.death_year != "-":
                if year <= int(self.death_year) and year >= int(self.birth_year):
                    flag =  True
            else:
                if year >= int(self.birth_year):
                    flag = True
        return flag
    
    def __repr__(self):
        return f"{self.name}, {self.birth_year}"

## B.2 Reading data

We have some data about a life events in a town, stored in a file `family_tree.txt`. The format is a chronological log with entries like either:

    year,id,born,name,mother_id,father_id
    
or

    year,id,died 
    
Because names can be repeated, every individual person has a unique ID in this data set. This is just a unique string.
    
For example, a part of the record might be:

    1650,u5019,born,Yatzil McGrue,u2013,u3031
    1651,u2013,died
    
This tells us "Yatzil McGrue" was born in 1650, and had parents with IDs u2013 and u3031. Yatzil's mother died in 1651.

When parent information is not known (e.g. from an immigrant into the town), it is recorded as "---". 

Read this data in using a function `read_family(fname)` that takes a filename and returns a dictionary mapping IDs (like `u2041`) to `Person` objects (using the class you defined above). 

In [9]:
## Solution
def read_family(filename):
    with open(filename) as f:
        l = f.read().splitlines()
        
    family_tree = {}
    death_lines = []
    
    for elt in l:
        line = elt.split(',')
        
        if len(line) > 3:
            birth_year = line[0]
            unique_id = line[1]
            name = line[3]
            mother_id = line[4]
            father_id = line[5]

            family_tree[unique_id] = Person(unique_id, name, birth_year, "-", mother_id, father_id)
        else:
            death_lines.append(line)
            
    for elt in death_lines:
        family_tree[elt[1]].death_year = elt[0]
        
    return family_tree

In [44]:
## Test
tree = read_family("family_tree.txt")

## B.3 Simple queries

(a) Write a *pure* function `census(tree, year)` that takes a dictionary like the one returned above, and a year, and returns a list of the people alive in that year. 

For example:
    
    census(tree, 1940)
    
    >>> [Meriel Seaver-Corey born 1851,
         Marlon Seaver-Corey born 1855,
         Crew Ewartborn born 1856,
         Brannon Noel-Ewart born 1856,
         ...
         Darian Ewart-Foster born 1940,
         Flanagan Falconer born 1940,
         Eliot Mills born 1940,
         Malone Mills born 1940]

In [8]:
## Solution
def census(tree, year):
    alive = []
    for person in tree:
        if tree[person].is_alive_in_particular_year(year):
            alive.append(f"{tree[person].name} born {tree[person].birth_year}")
    return alive


In [6]:
census(tree, 1940)

['Meriel Seaver-Corey born 1851',
 'Marlon Seaver-Corey born 1855',
 'Crew Ewart born 1856',
 'Brannon Noel-Ewart born 1856',
 'Gypsy Noel-Ewart born 1857',
 'Lee Seaver-Corey born 1859',
 'London Woodrow born 1859',
 'Stacie Woodrow born 1865',
 'Sondra Ackerman born 1869',
 'Darren Seaver-Corey born 1871',
 'Leanna Seaver-Corey born 1872',
 'Margo Woodrow born 1873',
 'Crew Ackerman born 1874',
 'Clint Corey born 1874',
 'Elisa Seaver-Corey born 1874',
 'Ian Ewart born 1874',
 'Brannon Noel-Ewart born 1874',
 'Merlyn Ackerman born 1875',
 'Caelan Woodrow born 1875',
 'Shannon Seaver-Corey born 1875',
 'Joselyn Britton born 1875',
 'Candyce Ackerman born 1876',
 'Tamika Britton born 1876',
 'Shayne Noel-Ewart born 1877',
 'Hepsie Noel-Ewart born 1878',
 'London Woodrow born 1879',
 'Gypsy Noel-Ewart born 1879',
 'Bethel Yates born 1879',
 'Gwenda Ackerman born 1880',
 'Shannon Mills born 1881',
 'Gordon Noel-Ewart born 1883',
 'Candyce Noel-Ewart born 1884',
 'Chasity Foster born 1884


(b) Write a function `roll(people, year)` that takes a list of people, and returns a string like the following, listing the name and age of each person in the list, in order, sorted by last name:

    Yatzil McGrue, age 34
    Imhotep Jones, age 69
    Ad-Habip Smith, age 19

Verify you can call it on the result returned from `census` and get a correct looking string returned, similar to below.
    
    roll(census(tree, 1940), 1940)

    >>> Sondra Ackerman, age 71
        Crew Ackerman, age 66
        Merlyn Ackerman, age 65
        Candyce Ackerman, age 64
        Gwenda Ackerman, age 60
        ...
        Kathlyn Yates, age 44
        Candyce Yates, age 24
        Gordon Yates, age 23
        Shannon Yates, age 18

In [7]:
## Solution
def roll(people, year):
    string_builder = ""
    sorted_people = []
    
    for element in people:
        line = element.split(' ')
        sorted_people.append(line)
    
    for element in sorted(sorted_people, key = lambda x: x[3]):
        name = element[0] + ' ' + element[1]
        birth_year = int(element[3])
        string_builder += name + ', ' + "age " + str(year - birth_year) + "\n"
    
    return string_builder
        

In [14]:
print(roll(census(tree, 1940), 1940))

Meriel Seaver-Corey, age 89
Marlon Seaver-Corey, age 85
Crew Ewart, age 84
Brannon Noel-Ewart, age 84
Gypsy Noel-Ewart, age 83
Lee Seaver-Corey, age 81
London Woodrow, age 81
Stacie Woodrow, age 75
Sondra Ackerman, age 71
Darren Seaver-Corey, age 69
Leanna Seaver-Corey, age 68
Margo Woodrow, age 67
Crew Ackerman, age 66
Clint Corey, age 66
Elisa Seaver-Corey, age 66
Ian Ewart, age 66
Brannon Noel-Ewart, age 66
Merlyn Ackerman, age 65
Caelan Woodrow, age 65
Shannon Seaver-Corey, age 65
Joselyn Britton, age 65
Candyce Ackerman, age 64
Tamika Britton, age 64
Shayne Noel-Ewart, age 63
Hepsie Noel-Ewart, age 62
London Woodrow, age 61
Gypsy Noel-Ewart, age 61
Bethel Yates, age 61
Gwenda Ackerman, age 60
Shannon Mills, age 59
Gordon Noel-Ewart, age 57
Candyce Noel-Ewart, age 56
Chasity Foster, age 56
Cullen Kimberly, age 56
Sienna Britton, age 56
Cletus Mills, age 56
Louis Mills, age 55
Meriel Ackerman-Fay, age 55
Eldon Ackerman, age 55
Tawnee Noel-Ewart, age 54
Leanna Ackerman-Fay, age 54
Me

In [13]:
print(roll(census(tree, 1940), 1940))

Meriel Seaver-Corey
Marlon Seaver-Corey
Crew Ewart
Brannon Noel-Ewart
Gypsy Noel-Ewart
London Woodrow
Joselyn Britton
Gwenda Ackerman
Trey Falconer
Shannon Yates
Genie Pound
Darian Teel
Candyce Ewart
Shayne Ackerman
Meriel Ewart
Tisha Falconer
Jenny Seaver-Corey
Drake Ewart
Chasity Mills
Debby Woodrow-Corey
Elisa Ewart
Desirae Langley-Langley
Carran Ewart-Foster
Maurine Turner
Allyn Falconer
Jamison Ewart
Gwenda Ewart
Gwenda Mills
Braiden Ewart
Ellington Ackerman-Ackerman
Cali Falconer
Merlyn Adams
Tisha Ewart
Crew Ewart
Ibbie Ackerman
Kassandra Woodrow-Corey
Margo Ewart
Genie Ackerman
Trey Mills
Sienna Ackerman
Allyn Falconer
Allyn Ewart
Beckham Woodrow-Corey
Gwenda Noel-Ewart
Flanagan Falconer
Eliot Mills
Malone Mills
None


(c) Write a function `find_by_name(people, name, year=None)` that will find all the people with the given name. If `year` is specified, only return those people who were alive in that year. Return the list of people in order of birth, earliest first.

**Sample output**:

```Python3
find_by_name(tree, "Bethel Wright")
>>> [Bethel Wright born 1650, Bethel Wright born 1850]
```

```Python3
find_by_name(tree, "Sonia Bond", 1700)
>>> [Sonia Bond born 1650, Sonia Bond born 1688]
```

```Python3
find_by_name(tree, "Debby Boon")
>>> [Debby Boon born 1650, Debby Boon born 1832, Debby Boon born 1833, Debby Boon born 1894]
```

```Python3
find_by_name(tree, "Tahnee Gill", 1725)
>>> [Tahnee Gill born 1694, Tahnee Gill born 1724]
```

In [45]:
## Solution
def find_by_name(tree, name, year = None):
    result = []
    flag = True
    
    if year != None:
        flag = False
        
    for person in tree:
        if tree[person].name == name:
            if flag:
                result.append(tree[person])
            else:
                if tree[person].is_alive_in_particular_year(year):
                    result.append(tree[person])
    
    return result

In [46]:
#find_by_name(tree, "Sonia Bond", 1700)
#find_by_name(tree, "Bethel Wright")
#find_by_name(tree, "Debby Boon")
find_by_name(tree, "Tahnee Gill", 1725)

[Tahnee Gill, 1694, Tahnee Gill, 1724]

(d) Write a function `memorial(tree, year)` that returns a string listing everyone who has died prior to `year`, in ascending order of death date (i.e., oldest death date first), in a format like:

    Yatzil McGrue died 1678, age 74
    Tzecan Barston died 1679 age 56
    
giving the ages at the time of death.

**Sample output**:

```Python3
memorial(tree, 1700)
>>>
    Travis Hyde died 1650, age 0
    Tahnee Gill died 1650, age 0
    Jenny Kimberly died 1651, age 1
    Alexandrina Mills died 1651, age 1
    Harold Kimberly died 1651, age 1
    Orville Kimberly died 1651, age 1
    ...
    Lynsay Victors died 1699, age 49
    Lynsay Mills died 1699, age 15
    Hepsie Kimberly died 1699, age 3
    Tahnee Garrard died 1699, age 2
    Darryl Fay died 1699, age 1
```

In [47]:
## Solution
def memorial(tree, year):
    result = []
    string_builder = ""
    for element in tree:
        name = tree[element].name
        if tree[element].death_year != "-":
            death_year = int(tree[element].death_year)
        else:
            continue
        age = tree[element].age_according_to_year(death_year)
        
        if death_year < year:
            result.append([name, death_year, age])
    
    for element in sorted(result, key = lambda x: x[1]):
        string_builder += f"{element[0]} died {element[1]}, age {element[2]}\n"
        
    return string_builder

In [48]:
print(memorial(tree, 1700))

Travis Hyde died 1650, age 0
Tahnee Gill died 1650, age 0
Jenny Kimberly died 1651, age 1
Alexandrina Mills died 1651, age 1
Harold Kimberly died 1651, age 1
Orville Kimberly died 1651, age 1
Eldon Mills died 1651, age 1
Marlon Kimberly died 1651, age 1
Darryl Mills died 1651, age 1
Flanagan Noel died 1652, age 2
Odell Kimberly died 1652, age 2
Odell Kimberly died 1652, age 2
Kayden Starr died 1652, age 2
Trey Corey died 1652, age 1
Diggory Bishop died 1653, age 3
Leanna Derrickson died 1653, age 3
Alexandrina Harris died 1653, age 3
Bryanne Brinley died 1653, age 3
Lacey Mills died 1653, age 3
Braiden Ewart died 1653, age 3
Lacey Starr died 1653, age 3
Eliana Ewart died 1653, age 3
Carrol Starr died 1653, age 3
Tawnee Devine died 1653, age 2
Carrol Starr died 1653, age 2
Carmen Kimberly died 1653, age 2
Genie Corey died 1653, age 2
Drake Simons died 1653, age 1
Lacey Bryson died 1654, age 4
Trey Falconer died 1654, age 4
Allyn Starr died 1654, age 4
Joe Kimberly died 1654, age 4
Gwend

## B.4 Class-ify
A dictionary of `People` is fine to work with, but as things get more complicated it would be useful to collect these operations into a class. 

* Create a class `FamilyTree` that stores a dictionary like we have been using, and add the methods `census(year)` and `find_by_name(name, year)` to it. 
* Create a **class method** `load_from_file` that will return a `FamilyTree` from a file, by calling `read_family`. We will extend this class in the next problems.

<div class="alert alert-info">
    <b> CLASS METHODS: </b> Just as class variables are unique to every instance of a class, class methods are also unique to every instance of a class. To be precise, a class method can modify a class state that would apply across all of the instance of the class. See usage below.    
</div>


In [ ]:
# to create a class method

class SomeClass:
    # constructor
    def __init__(self):
        pass
    
    # class method
    @classmethod
    def class_method_name(cls):
        pass
        

In [2]:
## Solution
class FamilyTree:
    
    # constructor and class method has been added for you
    
    def __init__(self, tree):
        self.tree = tree
    
    def census(self,year):
        return census(self.tree, year)
    
    def find_by_name(self, name, year = None):
        return find_by_name(self.tree, name, year)
    
    @classmethod
    def load_from_file(cls, fname):
        return FamilyTree(read_family(fname))
    
   

## B.5 Family relations
Add three new methods to `FamilyTree`:
    
* `parents(person)` -- returns a pair of the parents of a Person. Note: this should return two Person objects, *not* two IDs!    Return None for parents that aren't known ("---" in the data).
* `siblings(person)` -- returns a list of siblings of a Person (empty if none)
* `children(person)` -- returns a list of the the children of a Person (empty if none)

**Test** that these work correctly (try and think of a smart way of doing this)

In [78]:
## Solution                              
class FamilyTree:
    
    # constructor and class method has been added for you
    
    def __init__(self, tree):
        self.tree = tree
    
    def census(self,year):
        return census(self.tree, year)
    
    def find_by_name(self, name, year = None):
        return find_by_name(self.tree, name, year)
    
    def parents(self, person):
        mother_id = None
        father_id = None
        
        for element in self.tree:
            if self.tree[element].name == person.name and self.tree[element].birth_year == person.birth_year:
                if self.tree[element].mother_id != "-" and self.tree[element].father_id != "-":
                    mother_id = self.tree[element].mother_id
                    father_id = self.tree[element].father_id
                else:
                    if self.tree[element].mother_id == "-":
                        mother_id = None
                        father_id = self.tree[element].father_id
                    elif self.tree[element].father_id == "-":
                        father_id = None
                        mother_id = self.tree[element].mother_id
                        
        if mother_id != None and father_id != None:
            return (self.tree[mother_id], self.tree[father_id])
        else:
            if mother_id == None:
                return (None, self.tree[father_id])
            elif father_id == None:
                return (self.tree[mother_id], None)
        
    def children(self, person):
        children = []
        
        for element in self.tree:
            if self.tree[element].mother_id == person.unique_id or self.tree[element].father_id == person.unique_id:
                children.append(self.tree[element])
        return children
                
        
    def siblings(self, person):
        parents = self.parents(person)
        mother = parents[0]
        father = parents[1]
        
        siblings = list(set().union(self.children(mother), self.children(father)))
        siblings.remove(person)
        
        return siblings
        
        
    @classmethod
    def load_from_file(cls, fname):
        return FamilyTree(read_family(fname))
    

In [79]:
f = FamilyTree.load_from_file("family_tree.txt")
f.census(1940)

['Meriel Seaver-Corey born 1851',
 'Marlon Seaver-Corey born 1855',
 'Crew Ewart born 1856',
 'Brannon Noel-Ewart born 1856',
 'Gypsy Noel-Ewart born 1857',
 'Lee Seaver-Corey born 1859',
 'London Woodrow born 1859',
 'Stacie Woodrow born 1865',
 'Sondra Ackerman born 1869',
 'Darren Seaver-Corey born 1871',
 'Leanna Seaver-Corey born 1872',
 'Margo Woodrow born 1873',
 'Crew Ackerman born 1874',
 'Clint Corey born 1874',
 'Elisa Seaver-Corey born 1874',
 'Ian Ewart born 1874',
 'Brannon Noel-Ewart born 1874',
 'Merlyn Ackerman born 1875',
 'Caelan Woodrow born 1875',
 'Shannon Seaver-Corey born 1875',
 'Joselyn Britton born 1875',
 'Candyce Ackerman born 1876',
 'Tamika Britton born 1876',
 'Shayne Noel-Ewart born 1877',
 'Hepsie Noel-Ewart born 1878',
 'London Woodrow born 1879',
 'Gypsy Noel-Ewart born 1879',
 'Bethel Yates born 1879',
 'Gwenda Ackerman born 1880',
 'Shannon Mills born 1881',
 'Gordon Noel-Ewart born 1883',
 'Candyce Noel-Ewart born 1884',
 'Chasity Foster born 1884

In [80]:
person = f.find_by_name("Joss Ewart")[-1]
print(person)
print("-"*20)
print("Parents: ",f.parents(person))
print("-"*20)
print("Children: ",f.children(person))
print("-"*20)
print("Siblings: ",f.siblings(person))

Joss Ewart, 1838
--------------------
Parents:  (Dannie Ackerman, 1819, Meade Ewart, 1794)
--------------------
Children:  [Odell Corey, 1858, Eliot Corey, 1859, Ibbie Corey, 1860, Shannon Corey, 1861, Allyn Corey, 1862, Eliot Corey, 1863, Darryl Corey-Ewart, 1864, Teagan Corey, 1865, Beckham Corey, 1866, Carmen Ackerman, 1868, Sondra Ackerman, 1869, Leanna Ackerman, 1870, Kayden Ackerman, 1871, Malone Ackerman, 1872, Desirae Ackerman, 1873, Bethel Ackerman, 1874]
--------------------
Siblings:  [Caelan Ackerman, 1839, Crew Ewart, 1856, Juniper Ewart, 1836, Retha Ewart, 1847, Hepsie Ewart, 1842, Leanna Ewart, 1846, ChloГ« Ewart, 1858, Diggory Ewart, 1857, Sonia Ewart, 1843, Cletus Ewart, 1845, Forest Ewart, 1848, Chasity Ewart, 1840, Marlon Ewart, 1837, Steven Ewart, 1844, Lynsay Ewart, 1850, Michael Ewart, 1841, Caelan Ewart, 1852, Ricky Ewart, 1851, Dannie Ewart, 1849, Tranter Paternoster, 1853, Michael Ewart, 1854, Silver Ewart, 1855]


## B.6 [Optional] Harder queries

Add the following methods. These methods require the use of *recursion* to solve the problem.

An **ancestor** is a parent, grandparent, great-grandparent and so on.
A **descendent** is a child, grandchild, and so on.
A **nearest common ancestor** between two people is the most direct ancestor that two people share. For example, the nearest common ancestor of a niece and uncle is the niece's grandparent. The nearest common ancestor of siblings is their parent.
    
* `is_ancestor(a, b)` returns True if a is an ancestor of b 
* `is_descendent(a, b)` returns True if a is a descendent of b 
* `nearest_common_ancestor(a,b)` returns a Person who is a nearest common ancestor of `a` and `b` (or None, if no such ancestor). There may be more than one nearest common ancestor, in which case just return any one.
* `related(a, b)` returns True if two people have a common ancestor


In [ ]:
## Solution
